In [1]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

In [2]:
# !conda create -n st python pandas tqdm
# !conda activate st
# !conda install pytorch>=1.6 cudatoolkit=11.0 -c pytorch
# !pip install simpletransformers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

!unzip /content/drive/MyDrive/data.zip

Mounted at /content/drive
Archive:  /content/drive/MyDrive/data.zip
  inflating: data.csv                


In [4]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 k

In [5]:
import pandas as pd
import torch
import numpy as np

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available! Running on GPU.')
else:
    device = torch.device('cpu')
    print('CUDA is not available. Running on CPU.')

# Set a random seed for reproducibility
SEED = 2137
torch.manual_seed(SEED)

# Clear any cached memory
torch.cuda.empty_cache()

# Check if CUDA is available
print('Is CUDA available?', torch.cuda.is_available())

CUDA is available! Running on GPU.
Is CUDA available? True


In [6]:
!nvidia-smi

Wed Mar 13 19:55:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [8]:
train_data = pd.read_csv("data.csv", header = 0)
train_data.head()


,text,rating
0,We went here with our kids for Xmas holiday an...,4
1,We have spent in this hotel our summer holiday...,4
2,I visited Hotel Baltic with my husband for som...,4
3,I've travelled quite a numbers of hotels but t...,4
4,We decided for this family holiday destination...,3


In [9]:
train_data['rating'].value_counts()

4    524906
3    277453
2    110328
1     46883
0     40430
Name: rating, dtype: int64

In [10]:
SPLIT_POINT =20000
LIMIT = 200000
valid_data = train_data.iloc[:SPLIT_POINT,:]
train_data = train_data.iloc[SPLIT_POINT:LIMIT,:]
print("Shape of new dataframes - {} , {}".format(valid_data.shape, train_data.shape))

Shape of new dataframes - (20000, 2) , (180000, 2)


In [11]:
train_data['rating'].value_counts()

4    96426
3    49643
2    19323
1     8109
0     6499
Name: rating, dtype: int64

In [12]:
valid_data['rating'].value_counts()

4    9717
3    5697
2    2498
1    1080
0    1008
Name: rating, dtype: int64

In [13]:
# za dużo klasy 4 i 3, użyję arbitralnie dobranych wag, aby model mógł się skupić na innych klasach przy uczeniu

WEIGHTS = [9, 8, 3, 1.5, 1]

In [14]:
from simpletransformers.classification import ClassificationModel

# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=5, weight = WEIGHTS, use_cuda = True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
model.train_model(train_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/360 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/22500 [00:00<?, ?it/s]

(22500, 0.9248418806586001)

In [25]:
predictions, raw_outputs = model.predict(["I would love to burn this place to the ground! The food was uneatable, please close this place.", "Best place to ever be in, 10 outta ten my man!"])
print(predictions, raw_outputs)

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[0 4] [[ 4.23828125  1.61425781 -0.49609375 -2.82226562 -2.80273438]
 [-2.390625   -3.41210938 -1.76660156  2.08789062  5.3125    ]]


In [17]:
# Make predictions on the training data
predictions, raw_outputs = model.predict(valid_data['text'].tolist())

# Evaluate the predictions
# Assuming 'labels' are the actual labels in your training data
from sklearn.metrics import classification_report
print(classification_report(valid_data['rating'], predictions))

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.70      0.60      0.65      1008
           1       0.44      0.48      0.46      1080
           2       0.56      0.54      0.55      2498
           3       0.57      0.60      0.59      5697
           4       0.82      0.81      0.82      9717

    accuracy                           0.69     20000
   macro avg       0.62      0.61      0.61     20000
weighted avg       0.69      0.69      0.69     20000



In [22]:
# SAVE MODEL

output_dir = "outputs/checkpoint-22500-epoch-1"
model.save_model(output_dir)

import os
import shutil
save_dir = "model_18_35_12_03_2024"

In [24]:
## EXPORT
save_dir = "model_21_57_13_03_2024_69ACC"

shutil.copytree(output_dir, "/content/drive/My Drive/" + save_dir)

'/content/drive/My Drive/model_21_57_13_03_2024_69ACC'

In [ ]:
## IMPORT

# Copy the model directory from Google Drive to Colab

imported_dir = "imported_model_the_one"

shutil.copytree("/content/drive/My Drive/" + save_dir, imported_dir)

from simpletransformers.classification import ClassificationModel

# Load the model
model = ClassificationModel('roberta', imported_dir, use_cuda=True)